In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: ignored

In [3]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

NameError: ignored

In [4]:
print("Total Params:", FRmodel.count_params())

NameError: ignored

In [0]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
        
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)), axis = -1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)), axis = -1)
    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.0))
    
    return loss

In [6]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

NameError: ignored

In [0]:
import csv

def add_to_database (img1,img2,img3,id):
    frame = plt.imread(img1, 1)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    database = pd.read_csv('database.csv',index_col=0)
    for (x, y, w, h) in faces:
        roi = frame[y:y + h, x:x + w]
        cropped_img = cv2.resize(roi, (96, 96))
        plt.imshow(cropped_img, cmap = 'Greys_r')
        encoding1 = img_to_encoding(cropped_img, FRmodel)
        
    frame = plt.imread(img2, 1)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    database = pd.read_csv('database.csv',index_col=0)
    for (x, y, w, h) in faces:
        roi = frame[y:y + h, x:x + w]
        cropped_img = cv2.resize(roi, (96, 96))
        plt.imshow(cropped_img, cmap = 'Greys_r')
        encoding2 = img_to_encoding(cropped_img, FRmodel)

    frame = plt.imread(img3, 1)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    database = pd.read_csv('database.csv',index_col=0)
    for (x, y, w, h) in faces:
        roi = frame[y:y + h, x:x + w]
        cropped_img = cv2.resize(roi, (96, 96))
        plt.imshow(cropped_img, cmap = 'Greys_r')
        encoding3 = img_to_encoding(cropped_img, FRmodel)
        
    encoding=np.concatenate((encoding1[0],encoding2[0],encoding3[0]) , axis=0)
    print(len(encoding))
    database.loc[id]=encoding
    database.to_csv('database.csv')
    print(id)
            
 

In [0]:
add_to_database("images/omar.jpg","images/try.jpg","images/try5.jpg",1500121) #elite
add_to_database("images/g1.jpg","images/g2.jpg","images/g3.jpg",1500122)    #ganz
add_to_database("images/m1.jpg","images/m2.jpg","images/m3.jpg",1500123)    #oksh
add_to_database("images/h1.jpg","images/h4.jpg","images/h3.jpg",1500125)    #hassan
add_to_database("images/s4.jpg","images/s5.jpg","images/s8.jpg",1500124)    #salma
#add_to_database("images/m1.jpg","images/m2.jpg","images/m3.jpg",1500126)    #hamed

#add_to_database("images/denver.jpg",1500122)
#add_to_database("images/nairobi.jpg",1500123)
#add_to_database("images/tokyo.jpg",1500124)

In [ ]:

def who_is_it(image_path, model):
    #Reading the image path that is passed to the function
    frame = plt.imread(image_path, 1)
    # Using the haarcascade to get the face coordinates in the image
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    #reading the database file which contains the encodings for each student image
    database = pd.read_csv('database.csv',index_col=0)
    identity = 0
    for (x, y, w, h) in faces:
        roi = frame[y:y + h, x:x + w]
        cropped_img = cv2.resize(roi, (96, 96))
        plt.imshow(cropped_img)
        #After preprocessing the face of each person in the image
        #We iterate into each face and get its encodings to use it in the comparison
        encoding = img_to_encoding(cropped_img, FRmodel)
        min_dist = 100
        for id , db_enc in database.iterrows():
            dist1 = np.linalg.norm(db_enc[:128] - encoding[0])
            dist2 = np.linalg.norm(db_enc[128:256] - encoding[0])
            dist3 = np.linalg.norm(db_enc[256:] - encoding[0])
            #Compare the face with Every student 3 image encodings in the database
            #Then take the minimum distance to identify the student
            dist_avg = min(dist1,dist2,dist3)
            print(dist1)
            print(dist2)
            print(dist3)
            print(id)
            if dist_avg < min_dist:
                min_dist = dist_avg
                identity = id

        if min_dist > 0.8:
            print("Not in the database." )
            print(min_dist)
        else:
            #Here we state the name or code of the student that we find
            print ("it's " + str(identity) + ", the distance is " + str(min_dist))
    